In [2]:
# Q1. Create a table called 'employees' with various constraints
CREATE TABLE employees(
    emp_id INTEGER PRIMARY KEY NOT NULL,
    emp_name TEXT NOT NULL,
    age INTEGER CHECK (age >= 18),
    email TEXT UNIQUE,
    salary DECIMAL DEFAULT 30000
);


[('employees',)]


# Q2. Purpose of Constraints and Data Integrity

Constraints are rules applied to table columns to enforce the correctness, validity, and consistency of data.

✅ They help maintain **data integrity** by:
- Preventing invalid data from being inserted.
- Ensuring relationships between tables remain valid.
- Enforcing uniqueness and non-nullability where required.

### Common Types of Constraints:
1. **NOT NULL** – Ensures a column cannot have NULL values.
2. **UNIQUE** – Ensures all values in a column are different.
3. **PRIMARY KEY** – A combination of NOT NULL and UNIQUE. Uniquely identifies each row.
4. **FOREIGN KEY** – Enforces a link between the data in two tables.
5. **CHECK** – Ensures a column's value satisfies a specific condition.
6. **DEFAULT** – Provides a default value if no value is specified.


# Q3. Why use NOT NULL? Can a Primary Key contain NULL?

### ❓ Why use `NOT NULL`:
- To ensure important fields are always filled (e.g., `name`, `email`, etc.).
- Prevents accidental insertion of incomplete records.

### ❓ Can a PRIMARY KEY contain NULL?
**No**, a primary key **cannot contain NULL values**.

#### ✅ Reason:
- The primary key must uniquely identify each row.
- A NULL represents an unknown value, which violates uniqueness and identity rules.

🔒 Hence, when defining a primary key, SQL automatically enforces both `NOT NULL` and `UNIQUE`.


-- Q4. Add or Remove Constraints on an Existing Table

-- ✅ Example: Adding a CHECK constraint to the 'salary' column
ALTER TABLE employees
ADD CONSTRAINT chk_salary_positive CHECK (salary > 0);

-- ❌ Example: Removing a constraint (you must know the constraint name)
-- Assume the constraint name is 'chk_salary_positive'
ALTER TABLE employees
DROP CONSTRAINT chk_salary_positive;


# Q5. Consequences of Violating Constraints

When a constraint is violated during `INSERT`, `UPDATE`, or `DELETE`, the database will **reject the operation** and throw an **error message**.

### 🔴 Examples of Constraint Violations:
1. Inserting NULL into a NOT NULL column
2. Duplicating values in a UNIQUE column
3. Inserting an age less than 18 in a CHECK-constrained column
4. Inserting a foreign key value that does not exist in the referenced table

### ⚠️ Sample Error Message:
```sql
ERROR:  new row for relation "employees" violates check constraint "employees_age_check"
DETAIL:  Failing row contains (101, John Doe, 15, john@example.com, 40000).


In [ ]:
# ✅ Importing necessary libraries
import sqlite3

# ✅ Connecting to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# ✅ Step 1: Create the products table without constraints
cursor.execute("""
CREATE TABLE products (
    product_id INT,
    product_name VARCHAR(50),
    price DECIMAL(10, 2)
);
""")

# ✅ Step 2: Alter the table to add PRIMARY KEY constraint to product_id
cursor.execute("""
ALTER TABLE products
ADD PRIMARY KEY (product_id);
""")

# ✅ Step 3: Alter the table to set a DEFAULT value of 50.00 for price
cursor.execute("""
ALTER TABLE products
ALTER COLUMN price SET DEFAULT 50.00;
""")

# ✅ Confirming the structure of the updated table
cursor.execute("PRAGMA table_info(products);")
for column in cursor.fetchall():
    print(column)

# ✅ Closing the connection
conn.close()


OperationalError: near "PRIMARY": syntax error

In [ ]:
# ✅ Import SQLite and create an in-memory database
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# ✅ Step 1: Create the students table
cursor.execute("""
CREATE TABLE students (
    student_id INT PRIMARY KEY,
    student_name TEXT,
    class_id INT
);
""")

# ✅ Step 2: Create the classes table
cursor.execute("""
CREATE TABLE classes (
    class_id INT PRIMARY KEY,
    class_name TEXT
);
""")

# ✅ Step 3: Insert sample data into students
cursor.executemany("""
INSERT INTO students (student_id, student_name, class_id)
VALUES (?, ?, ?);
""", [
    (1, 'Aarav', 101),
    (2, 'Diya', 102),
    (3, 'Rohan', 103)
])

# ✅ Step 4: Insert sample data into classes
cursor.executemany("""
INSERT INTO classes (class_id, class_name)
VALUES (?, ?);
""", [
    (101, 'Physics'),
    (102, 'Chemistry'),
    (103, 'Mathematics')
])

# ✅ Step 5: INNER JOIN query to fetch student_name and class_name
cursor.execute("""
SELECT students.student_name, classes.class_name
FROM students
INNER JOIN classes ON students.class_id = classes.class_id;
""")

# ✅ Step 6: Display the result
results = cursor.fetchall()
for row in results:
    print(f"Student: {row[0]}, Class: {row[1]}")

# ✅ Close the connection
conn.close()


Student: Aarav, Class: Physics
Student: Diya, Class: Chemistry
Student: Rohan, Class: Mathematics


In [ ]:
# ✅ Setup: Create tables
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name TEXT
);
""")

cursor.execute("""
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT
);
""")

cursor.execute("""
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name TEXT
);
""")

# ✅ Insert sample data
cursor.executemany("INSERT INTO customers VALUES (?, ?);", [
    (1, 'Aarav'),
    (2, 'Diya')
])

cursor.executemany("INSERT INTO products VALUES (?, ?);", [
    (101, 'Laptop'),
    (102, 'Phone'),
    (103, 'Tablet')
])

cursor.executemany("INSERT INTO orders VALUES (?, ?, ?);", [
    (201, 1, 101),
    (202, 2, 102)
])

# ✅ INNER JOIN + LEFT JOIN to show all products
cursor.execute("""
SELECT orders.order_id, customers.customer_name, products.product_name
FROM products
LEFT JOIN orders ON products.product_id = orders.product_id
LEFT JOIN customers ON orders.customer_id = customers.customer_id;
""")

for row in cursor.fetchall():
    print(row)


(201, 'Aarav', 'Laptop')
(202, 'Diya', 'Phone')
(None, None, 'Tablet')


In [ ]:
# ✅ Add sales table
cursor.execute("""
CREATE TABLE sales (
    sale_id INT PRIMARY KEY,
    product_id INT,
    amount DECIMAL
);
""")

# ✅ Insert sample sales data
cursor.executemany("INSERT INTO sales VALUES (?, ?, ?);", [
    (1, 101, 50000),
    (2, 101, 30000),
    (3, 102, 20000)
])

# ✅ Query total sales per product
cursor.execute("""
SELECT products.product_name, SUM(sales.amount) AS total_sales
FROM products
INNER JOIN sales ON products.product_id = sales.product_id
GROUP BY products.product_name;
""")

for row in cursor.fetchall():
    print(row)


('Laptop', 80000)
('Phone', 20000)


In [ ]:
# ✅ Add quantity column to orders
cursor.execute("ALTER TABLE orders ADD COLUMN quantity INT;")

# ✅ Update quantity values
cursor.execute("UPDATE orders SET quantity = 2 WHERE order_id = 201;")
cursor.execute("UPDATE orders SET quantity = 1 WHERE order_id = 202;")

# ✅ Final INNER JOIN query
cursor.execute("""
SELECT orders.order_id, customers.customer_name, orders.quantity
FROM orders
INNER JOIN customers ON orders.customer_id = customers.customer_id;
""")

for row in cursor.fetchall():
    print(row)

# ✅ Close connection
conn.close()


(201, 'Aarav', 2)
(202, 'Diya', 1)


In [ ]:
-- Q1: Identify the primary keys and foreign keys in Maven Movies DB
-- 🔑 Primary Keys:
--    - actor.actor_id
--    - customer.customer_id
--    - film.film_id
--    - rental.rental_id
-- 🔗 Foreign Keys:
--    - film_actor.actor_id → actor.actor_id
--    - film_actor.film_id → film.film_id
--    - rental.customer_id → customer.customer_id
--    - rental.inventory_id → inventory.inventory_id

-- 🎯 Difference:
-- Primary Key ensures uniqueness of each record.
-- Foreign Key links one table to another, enforcing referential integrity.



SyntaxError: invalid character '🔑' (U+1F511) (ipython-input-2483979036.py, line 2)

In [ ]:
-- Q2: List all details of actors
SELECT * FROM actor;


SyntaxError: illegal target for annotation (ipython-input-199256097.py, line 1)

In [ ]:
-- Q3: List all customer information from DB
SELECT * FROM customer;


SyntaxError: illegal target for annotation (ipython-input-1810890809.py, line 1)

In [ ]:
-- Q4: List different countries
SELECT DISTINCT country FROM country;


SyntaxError: illegal target for annotation (ipython-input-818989539.py, line 1)

In [ ]:
-- Q5: Display all active customers
SELECT * FROM customer
WHERE active = 1;


SyntaxError: illegal target for annotation (ipython-input-1148640446.py, line 1)

In [ ]:
-- Q6: List of all rental IDs for customer with ID 1
SELECT rental_id FROM rental
WHERE customer_id = 1;


SyntaxError: illegal target for annotation (ipython-input-4122798053.py, line 1)

In [ ]:
-- Q7: Display all the films whose rental duration is greater than 5
SELECT * FROM film
WHERE rental_duration > 5;


SyntaxError: illegal target for annotation (ipython-input-830748462.py, line 1)

In [ ]:
-- Q8: Total number of films whose replacement cost is > $15 and < $20
SELECT COUNT(*) AS total_films FROM film
WHERE replacement_cost > 15 AND replacement_cost < 20;


SyntaxError: illegal target for annotation (ipython-input-536656346.py, line 1)

In [ ]:
-- Q9: Display the count of unique first names of actors
SELECT COUNT(DISTINCT first_name) AS unique_first_names FROM actor;


SyntaxError: illegal target for annotation (ipython-input-635550024.py, line 1)

In [ ]:
-- Q10: Display the first 10 records from the customer table
SELECT * FROM customer
LIMIT 10;



SyntaxError: illegal target for annotation (ipython-input-2973767016.py, line 1)

In [ ]:
-- Q11: Display the first 3 customer records whose first name starts with ‘B’
SELECT * FROM customer
WHERE first_name ILIKE 'b%'
LIMIT 3;


SyntaxError: invalid character '‘' (U+2018) (ipython-input-3964546700.py, line 1)

In [ ]:
-- Q12: Display the names of the first 5 movies which are rated as ‘G’
SELECT title FROM film
WHERE rating = 'G'
LIMIT 5;


SyntaxError: invalid character '‘' (U+2018) (ipython-input-364822722.py, line 1)

In [ ]:
-- Q13: Find all customers whose first name starts with 'A'
SELECT * FROM customer
WHERE first_name ILIKE 'a%';


SyntaxError: illegal target for annotation (ipython-input-1760090190.py, line 1)

In [ ]:
-- Q14: Find all customers whose first name ends with 'A'
SELECT * FROM customer
WHERE first_name ILIKE '%a';


SyntaxError: illegal target for annotation (ipython-input-1271030363.py, line 1)

In [ ]:
  -- Q15: Display the list of first 4 cities which start and end with ‘A’
SELECT city FROM city
WHERE city ILIKE 'a%a'
LIMIT 4;


SyntaxError: invalid character '‘' (U+2018) (ipython-input-684913181.py, line 1)

In [ ]:
-- Q16: Find all customers whose first name has 'NI' in any position
SELECT * FROM customer
WHERE first_name ILIKE '%ni%';



SyntaxError: illegal target for annotation (ipython-input-523072366.py, line 1)

In [ ]:

SELECT * FROM customer
WHERE first_name ILIKE '_r%';


SyntaxError: invalid syntax (ipython-input-4062842962.py, line 1)

In [ ]:
-- Q18: Find all customers whose first name starts with 'A' and is at least 5 characters long
SELECT * FROM customer
WHERE first_name ILIKE 'a%' AND LENGTH(first_name) >= 5;


SyntaxError: illegal target for annotation (ipython-input-3417772070.py, line 1)

In [ ]:
-- Q19: Find all customers whose first name starts with 'A' and ends with 'O'
SELECT * FROM customer
WHERE first_name ILIKE 'a%o';


SyntaxError: illegal target for annotation (ipython-input-818652392.py, line 1)

In [ ]:
-- Q20: Get the films with 'PG' and 'PG-13' rating using IN operator
SELECT * FROM film
WHERE rating IN ('PG', 'PG-13');


SyntaxError: illegal target for annotation (ipython-input-1646829129.py, line 1)

In [ ]:
-- Q21: Get the films with length between 50 to 100 using BETWEEN operator
SELECT * FROM film
WHERE length BETWEEN 50 AND 100;


SyntaxError: illegal target for annotation (ipython-input-869228265.py, line 1)

In [ ]:


SELECT COUNT(*) AS total_rentals FROM rental;


SyntaxError: Invalid star expression (ipython-input-3077072110.py, line 1)

In [ ]:
-- Question 2:
-- Find the average rental duration (in days) of movies rented from the Sakila database
SELECT AVG(rental_duration) AS average_rental_duration FROM film;


SyntaxError: invalid syntax (ipython-input-535783995.py, line 1)

In [ ]:
-- Question 3:
-- Display the first name and last name of customers in uppercase
SELECT UPPER(first_name) AS first_name_upper,
       UPPER(last_name) AS last_name_upper
FROM customer;


SyntaxError: invalid syntax (ipython-input-956414663.py, line 1)

In [ ]:
-- Question 4:
-- Extract the month from the rental date and display it alongside the rental ID
SELECT rental_id,
       MONTH(rental_date) AS rental_month
FROM rental;


SyntaxError: invalid syntax (ipython-input-3761426789.py, line 1)

In [ ]:
-- Question 5:
-- Retrieve the count of rentals for each customer (display customer ID and the count)
SELECT customer_id,
       COUNT(*) AS rental_count
FROM rental
GROUP BY customer_id;


SyntaxError: invalid syntax (ipython-input-2740443227.py, line 1)

In [ ]:
-- Question 6:
-- Find the total revenue generated by each store
SELECT store_id,
       SUM(amount) AS total_revenue
FROM payment
GROUP BY store_id;


SyntaxError: invalid syntax (ipython-input-2921426699.py, line 1)

In [ ]:
-- Question 7:
-- Determine the total number of rentals for each category of movies
SELECT c.name AS category_name,
       COUNT(rental.rental_id) AS total_rentals
FROM rental
JOIN inventory ON rental.inventory_id = inventory.inventory_id
JOIN film ON inventory.film_id = film.film_id
JOIN film_category fc ON film.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
GROUP BY c.name;


SyntaxError: invalid syntax (ipython-input-49717744.py, line 1)

In [ ]:
-- Question 8:
-- Find the average rental rate of movies in each language
SELECT l.name AS language,
       AVG(f.rental_rate) AS average_rental_rate
FROM film f
JOIN language l ON f.language_id = l.language_id
GROUP BY l.name;


SyntaxError: invalid syntax (ipython-input-3035669216.py, line 1)

In [ ]:
-- Question 9:
-- Display the title of the movie, customer's first name, and last name who rented it

SELECT f.title AS movie_title,
       c.first_name,
       c.last_name
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN customer c ON r.customer_id = c.customer_id;


SyntaxError: unterminated string literal (detected at line 2) (ipython-input-1778669168.py, line 2)

In [ ]:
-- Question 10:
-- Retrieve the names of all actors who have appeared in the film "Gone with the Wind"

SELECT a.first_name,
       a.last_name
FROM film f
JOIN film_actor fa ON f.film_id = fa.film_id
JOIN actor a ON fa.actor_id = a.actor_id
WHERE f.title = 'Gone with the Wind';


SyntaxError: invalid syntax (ipython-input-1631345846.py, line 1)

In [ ]:
-- Question 11:
-- Retrieve the customer names along with the total amount they've spent on rentals

SELECT c.first_name,
       c.last_name,
       SUM(p.amount) AS total_spent
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name;


SyntaxError: unterminated string literal (detected at line 2) (ipython-input-3958957240.py, line 2)

In [ ]:
-- Question 12:
-- List the titles of movies rented by each customer in a particular city (e.g., 'London')

SELECT c.first_name,
       c.last_name,
       ci.city,
       f.title AS movie_title
FROM customer c
JOIN address a ON c.address_id = a.address_id
JOIN city ci ON a.city_id = ci.city_id
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE ci.city = 'London'
ORDER BY c.customer_id, f.title;


SyntaxError: invalid syntax (ipython-input-647914278.py, line 1)

In [ ]:
-- Question 13:
-- Display the top 5 rented movies along with the number of times they've been rented

SELECT f.title AS movie_title,
       COUNT(r.rental_id) AS times_rented
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id, f.title
ORDER BY times_rented DESC
LIMIT 5;


SyntaxError: unterminated string literal (detected at line 2) (ipython-input-3197469000.py, line 2)

In [ ]:
-- Question 14:
-- Determine the customers who have rented movies from both stores (store ID 1 and 2)

SELECT customer_id
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
GROUP BY customer_id
HAVING COUNT(DISTINCT i.store_id) = 2;


SyntaxError: invalid syntax (ipython-input-3901465366.py, line 1)

In [ ]:
-- Q1. Rank the customers based on the total amount they've spent on rentals

SELECT customer_id,
       first_name,
       last_name,
       SUM(amount) AS total_spent,
       RANK() OVER (ORDER BY SUM(amount) DESC) AS spending_rank
FROM customer
JOIN payment ON customer.customer_id = payment.customer_id
GROUP BY customer_id, first_name, last_name;


SyntaxError: unterminated string literal (detected at line 1) (ipython-input-3243086060.py, line 1)

In [ ]:
-- Q2. Calculate the cumulative revenue generated by each film over time

SELECT f.title,
       p.payment_date,
       SUM(p.amount) OVER (PARTITION BY f.film_id ORDER BY p.payment_date) AS cumulative_revenue
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN payment p ON r.rental_id = p.rental_id;


SyntaxError: invalid syntax (ipython-input-1222221567.py, line 1)

In [ ]:
-- Q3. Determine the average rental duration for each film, considering films with similar lengths

SELECT f.title,
       f.length,
       AVG(f.rental_duration) OVER (PARTITION BY f.length) AS avg_duration_for_length_group
FROM film f;


SyntaxError: invalid syntax (ipython-input-3393561898.py, line 1)

In [ ]:
-- Q4. Identify the top 3 films in each category based on their rental counts

SELECT c.name AS category_name,
       f.title,
       COUNT(r.rental_id) AS rental_count,
       RANK() OVER (PARTITION BY c.name ORDER BY COUNT(r.rental_id) DESC) AS category_rank
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY c.name, f.title
HAVING RANK() OVER (PARTITION BY c.name ORDER BY COUNT(r.rental_id) DESC) <= 3;


SyntaxError: invalid syntax (ipython-input-3125158693.py, line 1)

In [ ]:
-- Q5. Calculate the difference in rental counts between each customer's total rentals and the average

SELECT customer_id,
       COUNT(*) AS customer_rentals,
       ROUND(AVG(COUNT(*)) OVER (), 2) AS avg_rentals,
       COUNT(*) - ROUND(AVG(COUNT(*)) OVER (), 2) AS difference_from_avg
FROM rental
GROUP BY customer_id;


SyntaxError: unterminated string literal (detected at line 1) (ipython-input-1483992481.py, line 1)

In [ ]:
-- Q6. Find the monthly revenue trend for the entire rental store over time

SELECT DATE_TRUNC('month', payment_date) AS month,
       SUM(amount) AS monthly_revenue
FROM payment
GROUP BY DATE_TRUNC('month', payment_date)
ORDER BY month;


SyntaxError: invalid syntax (ipython-input-2241057701.py, line 1)

In [ ]:
-- Q7. Identify customers whose total spending falls within the top 20% of all customers

SELECT customer_id,
       first_name,
       last_name,
       total_spent
FROM (
    SELECT c.customer_id,
           c.first_name,
           c.last_name,
           SUM(p.amount) AS total_spent,
           NTILE(5) OVER (ORDER BY SUM(p.amount) DESC) AS spending_quintile
    FROM customer c
    JOIN payment p ON c.customer_id = p.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
) AS ranked_customers
WHERE spending_quintile = 1;


SyntaxError: invalid syntax (ipython-input-193868360.py, line 1)

In [ ]:
-- Q8. Calculate the running total of rentals per category, ordered by rental count

SELECT c.name AS category_name,
       f.title,
       COUNT(r.rental_id) AS rental_count,
       SUM(COUNT(r.rental_id)) OVER (PARTITION BY c.name ORDER BY COUNT(r.rental_id) DESC) AS running_total
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY c.name, f.title;


SyntaxError: invalid syntax (ipython-input-2441640741.py, line 1)

In [ ]:
-- Q9. Find films rented less than the average rental count in their category

WITH rental_stats AS (
    SELECT c.category_id,
           f.film_id,
           f.title,
           COUNT(r.rental_id) AS rental_count,
           AVG(COUNT(r.rental_id)) OVER (PARTITION BY c.category_id) AS avg_rental_count
    FROM film f
    JOIN film_category c ON f.film_id = c.film_id
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
    GROUP BY c.category_id, f.film_id, f.title
)
SELECT title,
       rental_count,
       avg_rental_count
FROM rental_stats
WHERE rental_count < avg_rental_count;


SyntaxError: invalid syntax (ipython-input-1749126393.py, line 1)

In [ ]:
-- Q10. Identify the top 5 months with the highest revenue and display revenue per month

SELECT TO_CHAR(payment_date, 'YYYY-MM') AS month,
       SUM(amount) AS monthly_revenue
FROM payment
GROUP BY TO_CHAR(payment_date, 'YYYY-MM')
ORDER BY monthly_revenue DESC
LIMIT 5;


SyntaxError: invalid syntax (ipython-input-1641890080.py, line 1)

-- 🔍 Definition:
1NF requires that:
  - Each column contains **atomic (indivisible)** values.
  - There are no **repeating groups** or arrays.

-- 📌 Example Violation:
Let’s consider a **hypothetical unnormalized version** of the `customer` table:

customer_id | name        | phone_numbers  
------------|-------------|-----------------
1           | John Smith  | 123456, 789101  
2           | Alice Jones | 987654

-- 🔧 Normalization to 1NF:
We split phone numbers into a separate table:

📌 New Table: `customer_phone`

customer_id | phone_number  
------------|--------------
1           | 123456  
1           | 789101  
2           | 987654  


-- 🔍 Definition:
2NF requires:
  - The table must already be in 1NF.
  - All **non-prime attributes** must depend on the **whole primary key**, not part of it.

-- 📌 Example: `film_category`
film_id | category_id | category_name

Here, if we use `(film_id, category_id)` as composite PK, then `category_name` depends only on `category_id`, not both → violates 2NF.

-- 🔧 To Normalize:
Separate `category_name` into a new `category` table:

✔️ Table: film_category (film_id, category_id)  
✔️ Table: category (category_id, category_name)


-- 🔍 Definition:
2NF requires:
  - The table must already be in 1NF.
  - All **non-prime attributes** must depend on the **whole primary key**, not part of it.

-- 📌 Example: `film_category`
film_id | category_id | category_name

Here, if we use `(film_id, category_id)` as composite PK, then `category_name` depends only on `category_id`, not both → violates 2NF.

-- 🔧 To Normalize:
Separate `category_name` into a new `category` table:

✔️ Table: film_category (film_id, category_id)  
✔️ Table: category (category_id, category_name)


-- 🔍 Definition:
3NF requires:
  - Table is in 2NF.
  - No **transitive dependencies** (non-key attribute depending on another non-key attribute).

-- 📌 Example Violation (Hypothetical):
payment table:
payment_id | customer_id | customer_name | amount

Here, `customer_name` depends on `customer_id` → transitive dependency.

-- 🔧 To Normalize:
Remove `customer_name` from payment table and get it by joining with `customer` table.


-- 🔍 UNNORMALIZED (Assume a merged table):
customer_id | name     | address            | city       | country
------------|----------|--------------------|------------|---------
1           | John     | 101 Main Street    | Toronto    | Canada

-- 🔁 Normalize to 1NF:
✔️ Split address components if any are grouped.

-- 🔁 Normalize to 2NF:
✔️ Move city and country to separate tables:

Table: country (country_id, country_name)  
Table: city (city_id, city_name, country_id)  
Table: address (address_id, address_text, city_id)

Table: customer (customer_id, name, address_id)


-- Display actor names and the number of films they’ve acted in using CTE
WITH actor_films AS (
  SELECT actor_id, COUNT(film_id) AS film_count
  FROM film_actor
  GROUP BY actor_id
)
SELECT a.first_name, a.last_name, af.film_count
FROM actor a
JOIN actor_films af ON a.actor_id = af.actor_id;


-- Show film title, language, and rental rate using a CTE
WITH film_lang AS (
  SELECT film_id, title, language_id, rental_rate
  FROM film
)
SELECT f.title, l.name AS language, f.rental_rate
FROM film_lang f
JOIN language l ON f.language_id = l.language_id;


-- CTE to calculate total amount paid by each customer
WITH customer_revenue AS (
  SELECT customer_id, SUM(amount) AS total_revenue
  FROM payment
  GROUP BY customer_id
)
SELECT c.first_name, c.last_name, cr.total_revenue
FROM customer c
JOIN customer_revenue cr ON c.customer_id = cr.customer_id;


-- CTE to rank films by rental duration (descending)
WITH ranked_films AS (
  SELECT film_id, title, rental_duration,
         RANK() OVER (ORDER BY rental_duration DESC) AS duration_rank
  FROM film
)
SELECT * FROM ranked_films;


-- Step 1: CTE to find customers with more than 2 rentals
WITH frequent_customers AS (
  SELECT customer_id, COUNT(*) AS rental_count
  FROM rental
  GROUP BY customer_id
  HAVING COUNT(*) > 2
)

-- Step 2: Join with customer table to get details
SELECT c.customer_id, c.first_name, c.last_name, fc.rental_count
FROM frequent_customers fc
JOIN customer c ON fc.customer_id = c.customer_id;


WITH MonthlyRentals AS (
    SELECT
        DATE_TRUNC('month', rental_date) AS rental_month,
        COUNT(*) AS total_rentals
    FROM rental
    GROUP BY DATE_TRUNC('month', rental_date)
)
SELECT * FROM MonthlyRentals
ORDER BY rental_month;


In [ ]:
WITH ActorPairs AS (
    SELECT
        fa1.actor_id AS actor1_id,
        fa2.actor_id AS actor2_id,
        fa1.film_id
    FROM film_actor fa1
    JOIN film_actor fa2
        ON fa1.film_id = fa2.film_id
       AND fa1.actor_id < fa2.actor_id
)
SELECT
    ap.film_id,
    a1.first_name || ' ' || a1.last_name AS actor_1,
    a2.first_name || ' ' || a2.last_name AS actor_2
FROM ActorPairs ap
JOIN actor a1 ON ap.actor1_id = a1.actor_id
JOIN actor a2 ON ap.actor2_id = a2.actor_id
ORDER BY ap.film_id, actor_1, actor_2;


SyntaxError: invalid syntax (ipython-input-3658564692.py, line 1)

In [ ]:
-- Example: Find all staff who report (directly or indirectly) to manager with staff_id = 1

WITH RECURSIVE StaffHierarchy AS (
    SELECT
        staff_id,
        first_name,
        last_name,
        reports_to
    FROM staff
    WHERE staff_id = 1  -- Manager ID

    UNION ALL

    SELECT
        s.staff_id,
        s.first_name,
        s.last_name,
        s.reports_to
    FROM staff s
    JOIN StaffHierarchy sh ON s.reports_to = sh.staff_id
)
SELECT * FROM StaffHierarchy;


SyntaxError: illegal target for annotation (ipython-input-1496189638.py, line 1)